In [ ]:
import collections
import matplotlib.pyplot as plt
import getpass
import glob
import itertools
import json
import numpy as np
import os
import openml
import pandas as pd
import scipy.stats

from plot_utils.style import style_dc
from plot_utils.common_plots import rank, average_plot
from plot_utils.common_tables import collect_data_for_final_table, do_wilcoxon_test
from plot_utils.common_loading import load_from_openml

import sys
sys.path.append("/home/eggenspk/Work/Project/2020_PoSH_Autosklearn/2020_IEEE_Autosklearn_experiments/experiment_scripts/")
sys.path.append("/home/feurerm/sync_dir/projects/2020_posh/2020_IEEE_Autosklearn_experiments/experiment_scripts")
from utils import openml_automl_benchmark, get_normalization_constants

In [ ]:
username = getpass.getuser()
dir_ = {
    'eggenspk': "/media/eggenspk/04a9389c-b7e2-474a-a9de-c66d5345f407/2020_posh/",
    'feurerm': "/home/feurerm/projects/2020_posh/",
}[username]

valid_pretty = {
    10: {
    (None, "10MIN/ASKL_automldata/RF/RF_None_holdout_iterative_es_if"): "holdout",
    (None, "10MIN/ASKL_automldata/RF/RF_SH-eta4-i_holdout_iterative_es_if"): "SH; holdout",
    (None, "10MIN/ASKL_automldata/RF/RF_None_3CV_iterative_es_if"): "3CV",
    (None, "10MIN/ASKL_automldata/RF/RF_SH-eta4-i_3CV_iterative_es_if"): "SH; 3CV",
    (None, "10MIN/ASKL_automldata/RF/RF_None_5CV_iterative_es_if"): "5CV",
    (None, "10MIN/ASKL_automldata/RF/RF_SH-eta4-i_5CV_iterative_es_if"): "SH; 5CV",
    (None, "10MIN/ASKL_automldata/RF/RF_None_10CV_iterative_es_if"): "10CV",
    (None, "10MIN/ASKL_automldata/RF/RF_SH-eta4-i_10CV_iterative_es_if"): "SH; 10CV",
    },
    60: {
    (None, "60MIN/ASKL_automldata/RF/RF_None_holdout_iterative_es_if"): "holdout",
    (None, "60MIN/ASKL_automldata/RF/RF_SH-eta4-i_holdout_iterative_es_if"): "SH; holdout",
    (None, "60MIN/ASKL_automldata/RF/RF_None_3CV_iterative_es_if"): "3CV",
    (None, "60MIN/ASKL_automldata/RF/RF_SH-eta4-i_3CV_iterative_es_if"): "SH; 3CV",
    (None, "60MIN/ASKL_automldata/RF/RF_None_5CV_iterative_es_if"): "5CV",
    (None, "60MIN/ASKL_automldata/RF/RF_SH-eta4-i_5CV_iterative_es_if"): "SH; 5CV",
    (None, "60MIN/ASKL_automldata/RF/RF_None_10CV_iterative_es_if"): "10CV",
    (None, "60MIN/ASKL_automldata/RF/RF_SH-eta4-i_10CV_iterative_es_if"): "SH; 10CV",
    },
    #36000: {
    #("auto", "10H/AutoAuto_simulate/dynamic/None/autoauto"): "dynamic",
    #("auto", "10H/AutoAuto_simulate/static/None/autoauto"): "single best",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_None_holdout_iterative_es_if"): "holdout",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_SH-eta4-i_holdout_iterative_es_if"): "SH; holdout",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_None_3CV_iterative_es_if"): "3CV",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_SH-eta4-i_3CV_iterative_es_if"): "SH; 3CV",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_None_5CV_iterative_es_if"): "5CV",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_SH-eta4-i_5CV_iterative_es_if"): "SH; 5CV",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_None_10CV_iterative_es_if"): "10CV",
    #(None, "10H/ASKL_run_with_portfolio/3600/RF/RF_SH-eta4-i_10CV_iterative_es_if"): "SH; 10CV",
    #},
}

task_ids = openml_automl_benchmark

res_dc = {}
miss = 0
for horizon in list(valid_pretty.keys()):
    res_dc[horizon] = {}
    for tid in task_ids:
        res_dc[horizon][tid] = {}
        for mode in list(valid_pretty[horizon].keys()):
            if type(mode) == tuple:
                auto, model_name = mode
            else:
                model_name = mode
                auto = None
            res_dc[horizon][tid][mode] = []
            for seed in range(10):
                fl_tmpl = dir_ + "/" + model_name + "_%d_%d_0_0/result.json" % (tid, seed)
                fl = glob.glob(fl_tmpl)               
                if len(fl) == 0:
                    miss += 1
                    print(fl_tmpl)
                    continue
                fl = fl[0]
                with open(fl, "r") as fh:
                    line = json.load(fh)
                    loss = line["0"]["trajectory"]
                    loss = [(l[0], l[1]) for l in loss]
                    loss = dict(loss)
                    res_dc[horizon][tid][mode].append(loss)
print("Missing %d entries" % miss)

In [ ]:
# Load some stuff from disc and openml - takes some time
tasks, task_ids_sorted_by_num_features = load_from_openml(task_ids)
min_diff_dc = get_normalization_constants(dir_, load=True)
tasks

In [ ]:
HORIZON = 10
for tid in task_ids_sorted_by_num_features:
    plt.figure(figsize=[16,12])
    colors = itertools.cycle(COLORS)
    for mode in list(valid_pretty[HORIZON].keys()):
        c = next(colors)
        tmp = pd.DataFrame(res_dc[HORIZON][tid][mode]).sort_index(axis=1).ffill(axis=1)
        med = tmp.median(axis=0)
        med.loc[HORIZON*60] = med.iloc[-1]
        low = tmp.quantile(0.25)
        low.loc[HORIZON*60] = low.iloc[-1]
        up = tmp.quantile(0.75, axis=0)
        up.loc[HORIZON*60] = up.iloc[-1]
        plt.plot(med.index, med.to_numpy(), label=valid_pretty[HORIZON][mode], linewidth=3)
        plt.fill_between(med.index, low, up, alpha=0.3)
    plt.title('Name: %s (%d), #instances: %d, #attributes: %d' % (
        tasks.loc[tid, 'name'], tid, tasks.loc[tid, 'NumberOfInstances'], tasks.loc[tid, 'NumberOfFeatures']))
    plt.legend(fontsize=20)
    plt.ylim([plt.ylim()[0], plt.ylim()[0] + 0.3*(plt.ylim()[1]-plt.ylim()[0])])
    #plt.yscale("log")
    plt.show()

In [ ]:
# Plot average BER across all datasets
HORIZON = 10
model_list = []
for m in valid_pretty[HORIZON].keys():
    model_list.append(m)
average_plot(model_list=model_list, res_dc=res_dc, valid_pretty=valid_pretty,
             horizon=HORIZON, task_ids_sorted_by_num_features=task_ids_sorted_by_num_features,
             min_diff_dc=min_diff_dc)
plt.yscale("log")
plt.show()

In [ ]:
# Plot average BER across all datasets
HORIZON = 60
model_list = []
for m in valid_pretty[HORIZON].keys():
    model_list.append(m)
average_plot(model_list=model_list, res_dc=res_dc, valid_pretty=valid_pretty,
             horizon=HORIZON, task_ids_sorted_by_num_features=task_ids_sorted_by_num_features,
             min_diff_dc=min_diff_dc)
plt.yscale("log")
plt.show()

In [ ]:
# Box plots
lims = {
    "robert": [0.53, 0.65],
    "fabert": [0.31, 0.37],
    "nomao": [0.033, 0.047],
}

for horizon in res_dc:
    model_list = sorted(list(valid_pretty[horizon].keys()), key=lambda x: "SH" in x[1])
    for tid in task_ids_sorted_by_num_features:
        losses = {}
        for m in model_list:
            data = res_dc[horizon][tid][m]
            assert len(data) == 10
            final_losses = []
            for traj in data:
                keys = np.sort(list(traj.keys()))
                l = traj[keys[-1]]
                final_losses.append(l)
            losses[m] = np.array(final_losses)
        labels = [valid_pretty[horizon][m] for m in model_list]
        plt.boxplot(
            [losses[m] for m in model_list[:4]],
            positions=np.arange(4),
            labels=labels[:4],
            boxprops={"linewidth": 2},
            whiskerprops={"linewidth": 2},
            capprops={"linewidth": 2},
            medianprops={"linewidth": 3, "c": style_dc["colors"][0]},
            flierprops={"marker": "+", "markersize": 10, "markeredgewidth": 2}
                   )
        plt.boxplot(
            [losses[m] for m in model_list[4:]],
            positions=np.arange(4, 8),
            labels=labels[4:],
            boxprops={"linewidth": 2},
            whiskerprops={"linewidth": 2},
            capprops={"linewidth": 2},
            medianprops={"linewidth": 3, "c": style_dc["colors"][1]},
            flierprops={"marker": "+", "markersize": 10, "markeredgewidth": 2}
                   )
        plt.xticks(rotation=90, fontsize=style_dc["fontsize"])
        plt.yticks(fontsize=style_dc["fontsize"])
        plt.ylabel("balanced error rate", fontsize=style_dc["fontsize"])
        title = "%dmin %s [%dx%d]"
        plt.title(title % (horizon, tasks.loc[tid, 'name'], tasks.loc[tid, 'NumberOfInstances'], 
                           tasks.loc[tid, 'NumberOfFeatures']),
                 fontsize=style_dc["fontsize"])
        if tasks.loc[tid, 'name'] in lims:
            plt.ylim(lims[tasks.loc[tid, 'name']])
        plt.show()

In [ ]:
# Assume we have either all or no models
horizon_list = sorted(list(valid_pretty.keys()))
model_list = {}
for h in horizon_list:
    model_list[h] = []
    for m in valid_pretty[h].keys():
        model_list[h].append(m)

tab_data, stat_test_data = collect_data_for_final_table(model_list, res_dc, valid_pretty, horizon_list,
                                                        task_ids_sorted_by_num_features, min_diff_dc)
not_different = do_wilcoxon_test(stat_test_data, model_list, horizon_list, valid_pretty, exclude=["oracle",])

In [ ]:
df = pd.DataFrame(tab_data)
#for horizon in horizon_list:
#    df['Rank_%s' % horizon] = df[horizon].rank(method='average', ascending=True)
print(df.to_latex())
print(not_different)

In [ ]:
HORIZON = 60
model_list = []
for m in valid_pretty[HORIZON].keys():
    if "oracle" in m[1]:
        continue
    model_list.append(m)
rank(model_list, res_dc, valid_pretty, HORIZON, task_ids_sorted_by_num_features, n_iter=200, steplength=5)